In [19]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm # Displays a progress bar

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, Subset, DataLoader, random_split
!pip install torchattacks
import torchattacks

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
# Load the dataset and train, val, test splits
print("Loading datasets...")
my_transformer = transforms.Compose([
    transforms.ToTensor(),
])
MNIST_train = datasets.MNIST('.', download=True, train=True, transform=my_transformer)
MNIST_test = datasets.MNIST('.', download=True, train=False, transform=my_transformer)


Loading datasets...


In [21]:
BATCH_SIZE = 100
trainloader = DataLoader(MNIST_train, batch_size=BATCH_SIZE, shuffle=True)
testloader = DataLoader(MNIST_test, batch_size=BATCH_SIZE)

In [22]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5),                                            
            nn.MaxPool2d(kernel_size=2),   
            nn.ReLU(),         
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2),                        
            nn.MaxPool2d(2),   
            nn.ReLU(),                
        )
        self.out = nn.Linear(1152, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)       
        output = self.out(x)
        return output

In [23]:
device = "cuda" if torch.cuda.is_available() else "cpu" # Configure device
model = Network()
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/bestModel_CW_2iters.pt'))


<All keys matched successfully>

In [24]:
# PGDl2 attack on CW model 

attack = torchattacks.PGDL2(model, eps=2.0, alpha=0.4, steps=10, random_start=True)
num_samples = 0
correct_prediction = 0

for image, target_label in tqdm(testloader):
  adv_images = attack(image, target_label)
  # print(type(image))
  # print(type(adv_images))
  #maybe introduce training here
  output = model(adv_images)
  finalPred = torch.argmax(output,dim=1)
  #check how many samples were predicted right
  correct_prediction += (finalPred.cuda() == target_label.cuda()).sum().item()
  num_samples += BATCH_SIZE 
  # adv_images=adv_images.to(device)
  # target_label=target_label.to(device)
  #train_step(attackedModel, adv_images, target_label)
  # perturbations.append(adv_images)
  # labels.append(target_label)

print("Number of Correct Predictions: " + str(correct_prediction))
print("Total Number of Samples: " + str(num_samples))
print()
accuracy = float(correct_prediction) / num_samples
print('Model Robust Accuracy: {:.3f}％'.format(accuracy * 100))
print('Successful Attack Accuracy: {:.3f}％'.format(100 - (accuracy * 100)))

100%|██████████| 100/100 [00:40<00:00,  2.46it/s]

Number of Correct Predictions: 1902
Total Number of Samples: 10000

Model Robust Accuracy: 19.020％
Successful Attack Accuracy: 80.980％
